In [1]:
import datetime

import polars as pl

from nwec.constants import PROJECT_ROOT

YEAR = 2023

# Number of Disconnections

In [2]:
df = pl.read_excel(f"{PROJECT_ROOT}/data/utility_reporting/avista/avista_{YEAR}.xlsx", sheet_id=2, has_header=False)

In [3]:
disconnects = df.select([df.columns[0]] + df.columns[2:15]).slice(1)
# Drop null columns and cast all columns to strings
null_columns = [col for col in disconnects.columns if disconnects[col][0] is None]
disconnects = disconnects.drop(null_columns)
disconnects = disconnects.with_columns([pl.col(col).cast(pl.Utf8) for col in disconnects.columns])
# Rename columns with proper headers
disconnects = disconnects.rename(disconnects.head(1).to_dicts()[0])
disconnects = disconnects.filter(pl.col("Zip Code").str.contains("Residential Total"))
disconnects = disconnects.with_columns([pl.col(col).cast(pl.Int32, strict=False) for col in disconnects.columns[2:]])
disconnects = disconnects.fill_null(0)

In [4]:
disconnects

Zip Code,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00
str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""Residential Totals""","""583""",806,1079,988,910,829,811,745,874,728,189,975


# Disconnection Notices

In [5]:
disconnect_notices = df.select([df.columns[0]] + df.columns[41:53]).slice(1)
# Drop null columns and cast all columns to strings
null_columns = [col for col in disconnect_notices.columns if disconnect_notices[col][0] is None]
disconnect_notices = disconnect_notices.drop(null_columns)
disconnect_notices = disconnect_notices.with_columns([pl.col(col).cast(pl.Utf8) for col in disconnect_notices.columns])
# Rename columns with proper headers
disconnect_notices = disconnect_notices.rename(disconnect_notices.head(1).to_dicts()[0])
disconnect_notices = disconnect_notices.filter(pl.col("Zip Code").str.contains("Residential Total"))
disconnect_notices = disconnect_notices.with_columns(
    [pl.col(col).cast(pl.Int32, strict=False) for col in disconnect_notices.columns[1:]]
)
disconnect_notices = disconnect_notices.fill_null(0)

In [6]:
disconnect_notices

Zip Code,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""Residential Totals""",6156,7039,8076,6259,5657,4585,3833,4638,4519,2999,4726,5373


# Fees - None

# Bill Assistance (Payment Arrangements)

In [7]:
df = pl.read_excel(f"{PROJECT_ROOT}/data/utility_reporting/avista/avista_{YEAR}.xlsx", sheet_id=4, has_header=False)

# Number of Customers

In [8]:
df = pl.read_excel(f"{PROJECT_ROOT}/data/utility_reporting/avista/avista_{YEAR}.xlsx", sheet_id=8, has_header=False)

In [9]:
is_num_customers_section = (
    df.head(1)
    .select(pl.col(col).str.contains(r"(?i)number of customers by customer class") for col in df.columns)
    .row(0)
)
num_customers_start_index = next(index for index, value in enumerate(is_num_customers_section) if value)
num_customers = df.select(df.columns[:2] + df.columns[num_customers_start_index : num_customers_start_index + 12])

In [10]:
new_columns = num_customers.select(num_customers.columns[2:]).slice(2, 2).to_dicts()[0]
for col in new_columns:
    if type(new_columns[col]) is str:
        new_columns[col] = datetime.datetime.strptime(new_columns[col], "%Y-%m-%d %H:%M:%S").astimezone(datetime.UTC)
    new_columns[col] = new_columns[col].strftime("%B %Y")
new_columns = {"column_1": "Zip Code", "column_2": "Customer Class"} | new_columns
num_customers = (
    num_customers.rename(new_columns)
    .filter(pl.col("Customer Class").str.contains(r"(?i)^residential$"))
    .drop("Customer Class")
)

In [11]:
num_customers = num_customers.with_columns(
    [pl.col(col).cast(pl.Float64, strict=False) for col in num_customers.columns[1:]]
)
num_customers = num_customers.with_columns(pl.col("Zip Code").cast(pl.Int64))
num_customers.sum()

Zip Code,January 2023,February 2023,March 2023,April 2023,May 2023,June 2023,July 2023,August 2023,September 2023,October 2023,November 2023,December 2023
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
10407700,25858.0,27463.0,27880.0,29804.0,28946.0,26189.0,28295.0,27303.0,29041.0,28220.0,27866.0,28616.0


# Arrearages

In [12]:
df = pl.read_excel(f"{PROJECT_ROOT}/data/utility_reporting/avista/avista_{YEAR}.xlsx", sheet_id=8, has_header=False)

In [13]:
is_arrearage_section = (
    df.head(1).select(pl.col(col).str.contains(r"(?i)past-due balances by customer class") for col in df.columns).row(0)
)
arrearage_start_index = next(index for index, value in enumerate(is_arrearage_section) if value)
arrearages = df.select(df.columns[:2] + df.columns[arrearage_start_index : arrearage_start_index + 48])

In [14]:
new_columns = arrearages.select(arrearages.columns[2:]).slice(1, 1).to_dicts()[0]
last_date = None
last_vintage = 30
for col in new_columns:
    if new_columns[col] is not None:
        date = datetime.datetime.strptime(new_columns[col], "%Y-%m-%d %H:%M:%S").astimezone(datetime.UTC)
        date = date.strftime("%B %Y")
        new_columns[col] = f"{date} - 30 Days"
        last_date = date
    elif last_vintage == 30:
        new_columns[col] = f"{last_date} - 60 Days"
        last_vintage = 60
    elif last_vintage == 60:
        new_columns[col] = f"{last_date} - 90 Days"
        last_vintage = 90
    else:
        new_columns[col] = f"{last_date} - Total Arrearages"
        last_vintage = 30
new_columns = {"column_1": "Zip Code", "column_2": "Customer Class"} | new_columns
arrearages = (
    arrearages.rename(new_columns)
    .filter(pl.col("Customer Class").str.contains(r"(?i)^residential$"))
    .drop("Customer Class")
)

In [15]:
arrearages = arrearages.with_columns([pl.col(col).cast(pl.Float64, strict=False) for col in arrearages.columns[1:]])
arrearages = arrearages.with_columns(pl.col("Zip Code").cast(pl.Int64))
arrearages.sum()

Zip Code,January 2023 - 30 Days,January 2023 - 60 Days,January 2023 - 90 Days,January 2023 - Total Arrearages,February 2023 - 30 Days,February 2023 - 60 Days,February 2023 - 90 Days,February 2023 - Total Arrearages,March 2023 - 30 Days,March 2023 - 60 Days,March 2023 - 90 Days,March 2023 - Total Arrearages,April 2023 - 30 Days,April 2023 - 60 Days,April 2023 - 90 Days,April 2023 - Total Arrearages,May 2023 - 30 Days,May 2023 - 60 Days,May 2023 - 90 Days,May 2023 - Total Arrearages,June 2023 - 30 Days,June 2023 - 60 Days,June 2023 - 90 Days,June 2023 - Total Arrearages,July 2023 - 30 Days,July 2023 - 60 Days,July 2023 - 90 Days,July 2023 - Total Arrearages,August 2023 - 30 Days,August 2023 - 60 Days,August 2023 - 90 Days,August 2023 - Total Arrearages,September 2023 - 30 Days,September 2023 - 60 Days,September 2023 - 90 Days,September 2023 - Total Arrearages,October 2023 - 30 Days,October 2023 - 60 Days,October 2023 - 90 Days,October 2023 - Total Arrearages,November 2023 - 30 Days,November 2023 - 60 Days,November 2023 - 90 Days,November 2023 - Total Arrearages,December 2023 - 30 Days,December 2023 - 60 Days,December 2023 - 90 Days,December 2023 - Total Arrearages
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
10407700,2.4621e6,1.3582e6,2.8349e6,6.6552e6,2.8895e6,2.0909e6,2.8826e6,7.8630e6,2.7780e6,2.5541e6,3.4246e6,8.7567e6,2478809.5,2.2073e6,4.1516e6,8.8377e6,1.6739e6,1.9885e6,4.199901e6,7.8623e6,1.1393e6,1.2488e6,3.5335e6,5.9216e6,1.1849e6,1.0682e6,3.2815e6,5534546.4,1.2527e6,1.0026e6,2.7728e6,5.0281e6,1.4520e6,1.1746e6,2.7268e6,5.3534e6,1.0732e6,1.1830e6,2.8353e6,5.0915e6,982703.64,968441.89,3.0957e6,5.0468e6,1.7480e6,1.1095e6,3.0320e6,5.8896e6


# KLI Arrearages
Same code as arrearages, just with a different search string

In [16]:
df = pl.read_excel(f"{PROJECT_ROOT}/data/utility_reporting/avista/avista_{YEAR}.xlsx", sheet_id=8, has_header=False)

In [17]:
is_kli_arrearage_section = (
    df.head(1)
    .select(pl.col(col).str.contains(r"(?i)past-due balances for known low-income household") for col in df.columns)
    .row(0)
)
kli_arrearage_start_index = next(index for index, value in enumerate(is_kli_arrearage_section) if value)
kli_arrearages = df.select(df.columns[:2] + df.columns[kli_arrearage_start_index : kli_arrearage_start_index + 48])

In [18]:
new_columns = kli_arrearages.select(kli_arrearages.columns[2:]).slice(1, 1).to_dicts()[0]
last_date = None
last_vintage = 30
for col in new_columns:
    if new_columns[col] is not None:
        date = datetime.datetime.strptime(new_columns[col], "%Y-%m-%d %H:%M:%S").astimezone(datetime.UTC)
        date = date.strftime("%B %Y")
        new_columns[col] = f"{date} - 30 Days"
        last_date = date
    elif last_vintage == 30:
        new_columns[col] = f"{last_date} - 60 Days"
        last_vintage = 60
    elif last_vintage == 60:
        new_columns[col] = f"{last_date} - 90 Days"
        last_vintage = 90
    else:
        new_columns[col] = f"{last_date} - Total Arrearages"
        last_vintage = 30
new_columns = {"column_1": "Zip Code", "column_2": "Customer Class"} | new_columns
kli_arrearages = (
    kli_arrearages.rename(new_columns)
    .filter(pl.col("Customer Class").str.contains(r"(?i)^residential$"))
    .drop("Customer Class")
)

In [19]:
kli_arrearages = kli_arrearages.with_columns(
    [pl.col(col).cast(pl.Float64, strict=False) for col in kli_arrearages.columns[1:]]
).fill_null(0)
kli_arrearages = kli_arrearages.with_columns(pl.col("Zip Code").cast(pl.Int64))
kli_arrearages.sum()

Zip Code,January 2023 - 30 Days,January 2023 - 60 Days,January 2023 - 90 Days,January 2023 - Total Arrearages,February 2023 - 30 Days,February 2023 - 60 Days,February 2023 - 90 Days,February 2023 - Total Arrearages,March 2023 - 30 Days,March 2023 - 60 Days,March 2023 - 90 Days,March 2023 - Total Arrearages,April 2023 - 30 Days,April 2023 - 60 Days,April 2023 - 90 Days,April 2023 - Total Arrearages,May 2023 - 30 Days,May 2023 - 60 Days,May 2023 - 90 Days,May 2023 - Total Arrearages,June 2023 - 30 Days,June 2023 - 60 Days,June 2023 - 90 Days,June 2023 - Total Arrearages,July 2022 - 30 Days,July 2022 - 60 Days,July 2022 - 90 Days,July 2022 - Total Arrearages,August 2022 - 30 Days,August 2022 - 60 Days,August 2022 - 90 Days,August 2022 - Total Arrearages,September 2022 - 30 Days,September 2022 - 60 Days,September 2022 - 90 Days,September 2022 - Total Arrearages,October 2022 - 30 Days,October 2022 - 60 Days,October 2022 - 90 Days,October 2022 - Total Arrearages,November 2022 - 30 Days,November 2022 - 60 Days,November 2022 - 90 Days,November 2022 - Total Arrearages,December 2022 - 30 Days,December 2022 - 60 Days,December 2022 - 90 Days,December 2022 - Total Arrearages
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
10407700,507564.97,301720.52,678301.81,1487587.3,823630.19,762010.28,854511.58,2.4402e6,497978.96,1.0390e6,1.4470e6,2.9840e6,500312.55,766460.57,2.0454e6,3.3122e6,334415.86,688545.01,2.1051e6,3.1280e6,193244.99,347350.26,1.5071e6,2047738.3,246529.24,293497.15,1.3677e6,1.9077e6,285887.86,333484.63,1.1360e6,1.7553e6,305000.39,424413.53,1.1756e6,1905002.1,218668.57,403189.22,1.3714e6,1.9933e6,161442.91,271281.63,1.4184e6,1.8511e6,285735.2,294593.43,1453869.5,2.0342e6
